In [1]:
# https://nitratine.net/blog/post/asymmetric-encryption-and-decryption-in-python/
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization # for creating pem

# private key
private_key = rsa.generate_private_key( 
    public_exponent=65537,
    key_size=2048,
    backend=default_backend()
)

# public_key
public_key = private_key.public_key()

In [2]:
# write private key to .pem file
pem_private = private_key.private_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PrivateFormat.PKCS8,
    encryption_algorithm=serialization.NoEncryption()
)

with open('private_key.pem', 'wb') as f:
    f.write(pem_private)

1704

In [3]:
# write public key to .pem file
pem_public = public_key.public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo
)

with open('public_key.pem', 'wb') as f:
    f.write(pem_public)

451

In [4]:
#open private key from .pem file
with open("private_key.pem", "rb") as key_file:
    private_key = serialization.load_pem_private_key(
        key_file.read(),
        password=None,
        backend=default_backend()
    )

In [5]:
#open public key from .pem file
with open("public_key.pem", "rb") as key_file:
    public_key = serialization.load_pem_public_key(
        key_file.read(),
        backend=default_backend()
    )

In [6]:
from cryptography.fernet import Fernet
sym_key=Fernet.generate_key()
sym_key

b'pEs5yqRDM97gb-yY44DrZcwf-sDy4EQZ5m1pwppNhSo='

In [7]:
#encrypt 
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding

encrypted = public_key.encrypt(
    sym_key,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)

In [8]:
#decrypt
original_message = private_key.decrypt(
    encrypted,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)
print(original_message)

b'pEs5yqRDM97gb-yY44DrZcwf-sDy4EQZ5m1pwppNhSo='


In [9]:
import base64
import os
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC

password_provided = "password" # This is input in the form of a string
password = password_provided.encode() # Convert to type bytes
salt = os.urandom(16) # CHANGE THIS - recommend using a key from os.urandom(16), must be of type bytes
kdf = PBKDF2HMAC(
    algorithm=hashes.SHA256(),
    length=32,
    salt=salt,
    iterations=100000,
    backend=default_backend()
)
key = base64.urlsafe_b64encode(kdf.derive(password)) # Can only use kdf once
print(key.decode("utf-8"))
print(sym_key)

37u1lJi8DkHjtDUnZUqHJwtoBJWYicEu_eO4nQm0dFY=
b'pEs5yqRDM97gb-yY44DrZcwf-sDy4EQZ5m1pwppNhSo='
